In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
if os.path.exists("../input/whats-cooking-kernels-only"):
    challenge_path='../input/whats-cooking-kernels-only'
else:
    challenge_path='../input'
mode = 'train' #'train', 'commit', 'submission'
test_size=0.1
# Any results you write to the current directory are saved as output.

In [ ]:
# Import the required libraries 

random_state = 7
import time
import nltk
from nltk.stem import WordNetLemmatizer
import re

from sklearn.preprocessing import LabelBinarizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, StratifiedShuffleSplit
from sklearn.utils import class_weight
from sklearn.preprocessing import StandardScaler
from scipy.sparse import csr_matrix
from scipy.sparse import hstack, csr_matrix

from keras import backend as K
from keras.models import Sequential, load_model, Model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam
from keras.utils.vis_utils import plot_model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate, Dropout, BatchNormalization,UpSampling2D,Concatenate, Dense, Activation,Flatten
from keras import regularizers

import numpy as np

import pandas as pd
import json
import pdb

def read_dataset(path):
    return json.load(open(path, encoding='utf-8')) 


In [ ]:
# Dataset Preparation
print ("Read Dataset ... ")

df_train_raw=pd.read_json(os.path.join(challenge_path,'train.json')).set_index('id')

df_train_raw['invalid']=[False] * (len(df_train_raw))
x_field='ingredients'
y_field='cuisine'
df_train_raw=df_train_raw.loc[df_train_raw.cuisine !=None]
df_train_raw.head()

In [ ]:
lemmatizer = WordNetLemmatizer()
def preprocess(ingredients):
    ingredients_text = ' '.join(ingredients)
    ingredients_text = ingredients_text.lower()
    ingredients_text = ingredients_text.replace('-', ' ')
    words = []
    for word in ingredients_text.split():
        if re.findall('[0-9]', word): continue
        if len(word) <= 2: continue
        if '’' in word: continue
        word = lemmatizer.lemmatize(word)
        if len(word) > 0: words.append(word)
    return ' '.join(words)
    
ing=df_train_raw[x_field].values.tolist()
cou=df_train_raw[y_field].values.tolist()

#tag to remove some invalid samples
find_duplicated=False
dup=0
texts=[]
for i in range(len(ing)):
    if df_train_raw['invalid'].values[i]==False:
        if len(ing[i])<1 or pd.isna(cou[i]):
            df_train_raw['invalid'].values[i]=True
        elif find_duplicated:
            if ing[i] in ing[i+1:]:
                dup+=1
                print("found dup at {}".format(i))
                df_train_raw['invalid'].values[i]=True
    newtext=preprocess(df_train_raw[x_field].values[i])
    texts.append(newtext)

df_train_raw['texts']=texts

In [ ]:
len_orig=[len(x) for x in df_train_raw[x_field].values.tolist()]
len_proc=[len(x.split(' ')) for x in texts]
df_train_raw['len_orig']=len_orig
df_train_raw['len_proc']=len_proc
df_train_raw.head()

In [ ]:
df_train=df_train_raw.loc[df_train_raw.invalid == False]
dup,len(df_train),len(df_train_raw)

In [ ]:
df_submission = pd.read_json(os.path.join(challenge_path,'test.json')).set_index('id')
df_submission.head()

In [ ]:
#let's read plain text before encoded
y_j=df_train[y_field].values
x_j=df_train['texts'].values

for i in range(len(df_submission)):
    df_submission[x_field].values[i]=preprocess(df_submission[x_field].values[i])
x_j_sub = df_submission[x_field].values

In [ ]:
y_lbr=LabelBinarizer()
y_j_conv=y_lbr.fit_transform(y_j)

y_classes=y_lbr.classes_
y_classes_transformed=y_lbr.transform(y_classes)

In [ ]:
#not used
class_weights = class_weight.compute_class_weight('balanced', y_classes, y_j)
class_weights={i:class_weights[i] for i in range(len(y_lbr.classes_))}
class_weights

In [ ]:
#vectorize texts

def tokenize(text):
    #print(text)
    tokens = nltk.word_tokenize(text)
    return tokens

x_cvr = TfidfVectorizer(analyzer='word', tokenizer=tokenize, token_pattern=None, stop_words='english') 

x_cvr.fit(x_j)
x_analyze = x_cvr.build_analyzer()

x_j_conv =x_cvr.transform(x_j).toarray()
x_j_sub_conv=x_cvr.transform(x_j_sub).toarray()

#x_cvr,x_j_conv.shape,x_j_sub_conv.shape

In [ ]:
xt,xv,yt,yv,yjt,yjv=train_test_split(x_j_conv,y_j_conv,y_j, test_size=test_size, stratify=y_j, random_state=7, shuffle=True)

In [ ]:
K.clear_session()

np.random.seed(7)

do=0.50

i = Input(shape=xt.shape[1:])
x=Dropout(do-0.1)(i)
x=Dense(640)(x) #, use_bias=False
x=BatchNormalization()(x)
x=Activation('relu')(x)
x=Dropout(do)(x)

o=Dense(yt.shape[1],activation='linear')(x)  
o=Dense(yt.shape[1],activation='softmax')(o)

model=Model(inputs=[i,], outputs=[o,])
model.summary()

In [ ]:
lr=0.005
epochs=100
verbose=2
model.compile(loss="categorical_crossentropy", optimizer=Adam(lr=lr), metrics=["accuracy"])

mi=''
if mode=="train":
    early_stopping = EarlyStopping(monitor='val_loss', patience=33, verbose=1)
    model_checkpoint2 = ModelCheckpoint("./best_loss{}.model".format(mi),monitor='val_loss', 
                                       save_best_only=True, verbose=1)
    model_checkpoint3 = ModelCheckpoint("./best_acc{}.model".format(mi),monitor='val_acc', 
                                       save_best_only=True, verbose=1)
    reduce_lr = ReduceLROnPlateau(factor=0.3, patience=7, min_lr=0.00001, verbose=verbose) 
    model.fit(x=[xt,],y=[yt]*1,validation_data=[[xv,],[yv]*1],callbacks=[early_stopping, model_checkpoint2, model_checkpoint3, reduce_lr], epochs=epochs, batch_size=256, shuffle=True, verbose=2)
elif mode == 'commit':
    early_stopping = EarlyStopping(monitor='loss', patience=19, verbose=1)
    model_checkpoint2 = ModelCheckpoint("./best_loss{}.model".format(mi),monitor='loss', 
                                       save_best_only=True, verbose=1)
    model_checkpoint3 = ModelCheckpoint("./best_acc{}.model".format(mi),monitor='acc', 
                                       save_best_only=True, verbose=1)
    reduce_lr = ReduceLROnPlateau(factor=0.3, patience=5, min_lr=0.00001, verbose=verbose) 
    model.fit(x=[x_j_conv,],y=[y_j_conv]*1,callbacks=[early_stopping, model_checkpoint2, model_checkpoint3, reduce_lr], epochs=epochs, batch_size=256, shuffle=True, verbose=2)
  

In [ ]:
if mode == 'train' or mode == 'commit':
    model_loss=load_model(os.path.join('.',"best_loss.model"))
    model_acc=load_model(os.path.join('.',"best_acc.model"))
elif mode=='submission':
    model_loss=load_model(os.path.join(user_path,"best_loss.model"))
    model_acc=load_model(os.path.join(user_path,"best_acc.model"))        
print(model_loss.evaluate([xv,],[yv]*1))
print(model_acc.evaluate([xv,],[yv]*1))

In [ ]:
y_j_sub_conv=model_loss.predict(x_j_sub_conv,verbose=1)
y_j_sub_conv2=model_acc.predict(x_j_sub_conv,verbose=1)

In [ ]:
y_j_sub=y_lbr.inverse_transform(y_j_sub_conv)
y_j_sub2=y_lbr.inverse_transform(y_j_sub_conv2)

In [ ]:
df_submission[y_field]= y_j_sub
df_submission.head()

# Submission
print ("Generate Submission File ... ")
sub=df_submission.drop(x_field,axis=1)
sub.head(50)
sub.to_csv('svm_output_loss.csv', index=True)
sub.to_csv('submission.csv', index=True)

with open('submission.csv', mode='r', encoding='utf-8') as f:
    csvtext=f.read()
print(csvtext)

In [ ]:
df_submission[y_field]= y_j_sub2
df_submission.head()

# Submission
print ("Generate Submission File ... ")
sub=df_submission.drop(x_field,axis=1)
sub.head(50)
sub.to_csv('svm_output_acc.csv', index=True)
sub.to_csv('submission.csv', index=True)

with open('submission.csv', mode='r', encoding='utf-8') as f:
    csvtext=f.read()
print(csvtext)